# Model Building and Training

In this notebook we will train our model on mixed data through multiple inputs.

In [1]:
import tarfile
import pickle
import pandas as pd
import bz2
from datetime import datetime,timedelta
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import cv2
import os
import glob


In [9]:
def save_pickle(file_name,obj):
    with open(file_name, 'wb') as fout:
        pickle.dump(obj, fout)

def open_pickle(file_name):
    with open(file_name, 'rb') as handle:
        obj = pickle.load(handle)
    return obj

def get_np_array_from_tar_object(tar_extractfl):
    '''converts a buffer from a tar file in np.array'''
    return np.asarray(bytearray(tar_extractfl.read())
                      , dtype=np.uint8)


def load_house_images(df,yr=None,tar=None):
    # initialize our images array (i.e., the house images themselves)
    images = []
    
    yr = '0'
    for img in df.values:
        yr_file = img[:4]
        if yr == yr_file:
            pass
        else:
            if tar:
                print("deleted tar")
                del tar
        
            yr = img[:4]
            file = f'data/Folsom_sky_images_{yr}.tar.bz2'
            print(yr,file)
            tar = tarfile.open(file)
        
        image = cv2.imdecode(get_np_array_from_tar_object(tar.extractfile(img)), 0)
        image = cv2.resize(image, (32, 32))
        images.append(image)
    
    return np.array(images)

### Read in Data:

In [3]:
model_data_dict = open_pickle('../data_rp/model_data_dict.pkl')
df_5_min = open_pickle('../data_rp/df_5_min_data.pkl')
df_10_min = open_pickle('../data_rp/df_10_min_data.pkl')
df_15_min = open_pickle('../data_rp/df_15_min_data.pkl')
df_20_min = open_pickle('../data_rp/df_20_min_data.pkl')
df_25_min = open_pickle('../data_rp/df_25_min_data.pkl')
df_30_min = open_pickle('../data_rp/df_30_min_data.pkl')

In [4]:
df_5_min = df_5_min['file']
df_10_min = df_10_min['file']
df_15_min = df_15_min['file']
df_20_min = df_20_min['file']
df_25_min = df_25_min['file']
df_30_min = df_30_min['file']

In [10]:
images_5_min = load_house_images(df_5_min)

2014 data/Folsom_sky_images_2014.tar.bz2


KeyboardInterrupt: 

In [ ]:
# model_data_dict.keys()

In [ ]:
# model_data_dict['df_5_min'].keys()

In [ ]:
tar0 = tarfile.open('data/Folsom_sky_images_2014.tar.bz2')

im0 = cv2.imdecode(get_np_array_from_tar_object(tar0.extractfile('2014/01/02/20140102_152808.jpg')), 0)

In [ ]:
from datetime import datetime

now = datetime.now()
# im1 = cv2.imdecode(get_np_array_from_tar_object(tar0.extractfile('2014/01/02/20140102_152907.jpg')), 0)
im2 = cv2.imdecode(get_np_array_from_tar_object(tar0.extractfile('2014/01/02/20140102_152907.jpg')), 0)
datetime.now() - now

In [ ]:
df_5_min.iloc[0][:4]

In [ ]:
new_img_file_names

In [ ]:
df_5_min.values

In [ ]:
tar_members_names